# Data Acquisition

## The Agricultural Data

<img src="Economic_Research_Service_Logo.PNG" height=400 width=400>

The agricultural yield and bearing acreage time series data was acquired from the United States Department of Agriculture's [Economic Research Service.](https://www.ers.usda.gov/data-products/fruit-and-tree-nut-data/fruit-and-tree-nut-yearbook-tables/) The USDA Fruit and Tree Nut Yearbook tables contain almost four decades of time-series data for U.S. bearing acreage, production, prices, etc. for a variety of Fruit and Nut Tree crops. We acquired our citrus crop data from a collection of these Fruit Tree Yearbook tables that look at bearing acreage, and yield per acre, for a time series ranging from 1980 through 2018. The citrus crop tables contained aggregated data for lemons, oranges, and grapefruit, for the states of California, Texas, Arizona, and Florida.

## The Weather Data

<img src="National_Climatic_Data_Center_logo.png">

The weather timeseries data was acquired from the NOAA's National Climatic Data Center, or NCDC for short. [An article on citrus production in California (page 2)](https://apps1.cdfa.ca.gov/FertilizerResearch/docs/Citrus_Production_CA.pdf) revealed that the counties of Kern, Fresno, Tulare, Ventura, and Riverside as being the top producers of citrus in the state of California, we decided to pull weather readings from stations in each of these five counties. The [Local Climatological Data](https://www.ncdc.noaa.gov/cdo-web/datatools/lcd) tool at the NCDC's website allowed us to pull hourly station readings, of temperature and relative humidity, from each of these five counties, spanning a time series from 2000 - 2018. The data came in the form of .csv files, which made them easy to process, but there was a significant delay between requesting the data, and when the data was actually served to us. Of that weather data, we decided to focus only on readings of hourly dry bulb temperature, and hourly relative humidity.

# Data Cleaning and Transformation

While the agricultural data gathered from the USDA was in .csv files, the organization of the data within these files was awkward, which necessitated the use of a custom python script to convert the data into rows that could easily be processed and analyzed.

While the agricultural data came in the form of a yearly time series of aggregate yield per acre and bearing acreage, the weather data consisted of raw station readings recorded at hourly intervals. Therefore, in order to match the weather data with the agricultural data on the same temporal scale, we needed to aggregate the hourly weather station readings into daily, monthly, and yearly summary statistics. We generated the following summary statistics for temperature and humidity.

- The average, min, and max temperature and relative humidity readings for each year.
- Temperature range (max - min)
- Number of days with temperatures 10 degrees above the yearly average, by year
- Number of days with temperatures 10 degrees below the yearly average, by year
- Number of days with minimum temperatures below 32 degrees, by year
- Number of days with minimum temperatures within 3 degrees of the monthly minimum, in months with extreme lows, by year
- Number of days with temperatures 7 degrees above the monthly average, by year
- Number of days with temperatures 7 degrees below the monthly average, by year
- Number of days with relative humidity 15% or more above the monthly average, by year
- Number of days with relative humidity 25% or more above the yearly average

These summary statistics were generated using a combination of SQL queries to generate the original yearly, monthly, and daily aggregates, and python list comprehensions to take care of the more complicated metrics.

After having generated these yearly aggregate statistics for weather, we then joined the agricultural data with the weather data into pandas DataFrames objects. Working with DataFrames gave us easy to use snapshots of our data, and made generating plots and correlation statistics of the various columns both easy and intuitive.

![Pandas DataFrame](pd_data_frame.png)

*Example DataFrame generated for lemons*

# Building Models

During our literature review, we happened upon [a 2010 article](https://core.ac.uk/download/pdf/6550507.pdf) which suggests that bearing acreage and yield for fresh grapefruit and oranges are partially dependent upon climate. 

In particular, the article presents the following models for bearing acreage and yield for fresh oranges and grapefruit, which we have summarized below.

$Yield_t = f(Climate_t, Prices_{t - 1}, Technology_t, LandQuality_t)$

$Acreage_t = f(Acreage_{t - 1}, Acreage_{t - 2}, Climate_{t - 1}, Prices_{t - 1}, ...)$

Without focusing too much on the details, we noticed that both of these models include climate as an independent variable. We also noted that while the yield for a crop at timestep $t$ is dependent upon the climatic conditions at that same time step, the acreage is dependent upon the conditions at timestep $t - 1$, indicating a lag in the effects of climate on bearing acreage of citrus crops.